# Data Exploration

In this notebook, we load the KITTI dataset and explore the data.

In [ ]:
# Import statements.
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from typing import Dict, Any, List, Optional, Tuple

In [ ]:
# Global constants.
KITTI_ROOT_DIR = os.path.join('..', 'data', 'KITTI')
LEFT_CAM_ROOT_DIR = os.path.join(KITTI_ROOT_DIR, 'data_object_image_2')
TRAIN_IMAGE_DIR = os.path.join(LEFT_CAM_ROOT_DIR, 'training', 'image_2')
TEST_IMAGE_DIR = os.path.join(LEFT_CAM_ROOT_DIR, 'testing', 'image_2')
LABEL_DIR = os.path.join(KITTI_ROOT_DIR, 'training', 'label_2')
DEFAULT_DATASET_ARGS = {'val_split': 0.3}
TRAIN_KEY = 'train'
VAL_KEY = 'val'
TEST_KEY = 'test'

In [ ]:
def get_partition(train_image_dir: str,
                  test_image_dir: str, dataset_args: Dict[str, Any] =
                  DEFAULT_DATASET_ARGS) -> Dict[str,List[str]]:
    """Returns a dict where the keys are 'train', 'test', and 'val', and the
    values are the images under each. The list is the authoratative order of the
    train/test examples; partition['train'][0] is the first training example,
    and x_train[0] will correspond with that filename.
    :param train_image_dir: The directory in which are located all the training
    images.
    :param test_image_dir: The directory in which are located all the test
    images.
    :param dataset_args: The dataset arguments. See DEFAULT_DATASET_ARGS for
    available options.
    :return: The train/val/test partition.
    """
    dataset_args = {**DEFAULT_DATASET_ARGS, **dataset_args}
    partition = {}
    train_image_filenames = [filename for
                             filename in os.listdir(train_image_dir) if
                             filename.endswith('.png')]
    rand_indices = np.random.permutation(len(train_image_filenames))
    split_index = int(dataset_args['val_split'] * len(train_image_filenames))
    val_indices = rand_indices[:split_index]
    train_indices = rand_indices[split_index:]
    partition[TRAIN_KEY] = [train_image_filenames[i] for i in train_indices]
    partition[VAL_KEY] = [train_image_filenames[i] for i in val_indices]
    test_image_filenames = [filename for
                            filename in os.listdir(test_image_dir) if
                            filename.endswith('.png')]
    partition[TEST_KEY] = test_image_filenames
    return partition

In [ ]:
def load_image_into_numpy_array(
    path: str,
    target_size: Optional[Tuple[int, int]] = None) -> np.ndarray:
    """Load an image from file into a numpy array.
    :param path: The path to the image.
    :param target_size: If specified, the width and height of the output array.
    If None, the image is size is unchanged.
    :return: The image contents as an np.ndarray of type uint8.
    """
    img_data = cv2.imread(path)
    img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
    if target_size:
        img_data = cv2.resize(img_data, target_size)
    return img_data.astype(np.uint8)

In [ ]:
def display_image(filename: str,
                  fig_size: Tuple[float, float] = (9.0, 2.75)) -> None:
    """Displays the image at filename.
    :param filename: The path to the image.
    :param fig_size: The size of the output, in inches. The default values will
    show the KITTI images in the correct aspect ratio without passing the 80
    character vertical margin in jupyter.
    """
    plt.imshow(load_image_into_numpy_array(filename))
    fig = plt.gcf()
    fig.set_size_inches(*fig_size)
    plt.tight_layout()

In [ ]:
def display_image_2d_boxes(filename: str,
                           image_labels: List[str],
                           fig_size: Tuple[float, float] = (9.0, 2.75)) -> None:
    """Displays the image at filename with its 2D bounding boxes.
    :param filename: The path to the image.
    :param image_labels: The image's labels.
    :param fig_size: The size of the output, in inches.
    """
    display_image(filename, fig_size=fig_size)
    ax = plt.gca()
    for line in image_labels:
        x_left, y_top, x_right, y_bot = get_label_2d_box(line)
        width = x_right - x_left
        height = y_bot - y_top
        rect = patches.Rectangle((x_left, y_top), width, height, linewidth=3,
                                 edgecolor='g', facecolor='none')
        ax.add_patch(rect)

In [ ]:
def display_image_3d_boxes(filename: str,
                           image_labels: List[str],
                           fig_size: Tuple[float, float] = (9.0, 2.75)) -> None:
    """Displays the image at filename with its 3D bounding boxes.
    :param filename: The path to the image.
    :param image_labels: The image's labels.
    :param fig_size: The size of the output, in inches.
    """
    display_image(filename, fig_size=fig_size)
    ax = plt.gca()
    for line in image_labels:
        # TODO get 3D box and draw it.
        # See https://github.com/smallcorgi/3D-Deepbox/tree/master/visualization
        # computeBox3D.m and then projectToImage.m
        pass

In [ ]:
def get_label_class(label_line: str) -> str:
    """Returns the class from the line from a label file.
    :param label_line: A line from a label file.
    :return: The class name.
    """
    return label_line.split(' ')[0]

In [ ]:
def get_label_2d_box(label_line: str) -> (float, float, float, float):
    """Returns the 2D bounding box from the line from a label file.
    :param label_line: A line from a label file.
    :return: The 2D bounding box as left, top, right, bottom.
    """
    return tuple([float(num) for num in label_line.split(' ')[4:8]])

In [ ]:
def get_labels(label_dir: str) -> Dict[str, List[str]]:
    """Returns a dict where the keys are the image filenames and the values are
    the labels. Each file has several labels, each of which represents an
    object.
    :param label_dir: The directory containing the ground truth label files.
    :return: The label dict.
    """
    labels = {}
    label_filenames = [filename for filename in os.listdir(label_dir) if
                       filename.endswith('.txt')]
    for filename in label_filenames:
        with open(os.path.join(label_dir, filename), 'r') as infile:
            labels[filename.replace('.txt', '.png')] = \
                [line.strip() for line in infile.readlines()]
    return labels

In [ ]:
partition = get_partition(TRAIN_IMAGE_DIR, TEST_IMAGE_DIR)
labels = get_labels(LABEL_DIR)
print('{0} train images'.format(len(partition[TRAIN_KEY])))
print('{0} val images'.format(len(partition[VAL_KEY])))
print('{0} test images'.format(len(partition[TEST_KEY])))
print('{0} labels'.format(len(labels.keys())))

In [ ]:
display_image(os.path.join(TRAIN_IMAGE_DIR, partition[TRAIN_KEY][0]))

In [ ]:
# Use image 0000008.png for consistency (partition is in random order).
example_filename = '000008.png'
display_image(os.path.join(TRAIN_IMAGE_DIR, example_filename))
print('Labels:')
for label in labels[example_filename]:
    print(label)

In [ ]:
display_image_2d_boxes(os.path.join(TRAIN_IMAGE_DIR, example_filename),
                      labels[example_filename])